In [1]:
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
node = sy.orchestra.launch(name="test", reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

Logged into <test: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
from syft.service.code.user_code import execute_byte_code

In [4]:
@sy.syft_function_single_use()
def test_func():
    import time
    print("Start")
    time.sleep(1)
    return 1

SyftSuccess: Syft function 'test_func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
domain_client.code.request_code_execution(test_func)

syft.service.request.request.Request

In [6]:
domain_client.requests[-1].approve()

Request approved for domain test


SyftSuccess: Request a90d40d160a6443ca68cbb9b2203bb6e changes applied

In [7]:
print(domain_client.code[0].parsed_code)

def user_func_test_func_67d3b5eaf0c0bf6b5a602d359daecc86a7a74053490ec37ae08e71360587c870_530c8340dd99ba84b9533f27a22b96782da820ff5b391782cdd0c5bae62dd9f0():

    def test_func():
        import time
        logger.print('Start')
        time.sleep(1)
        return 1
    result = test_func()
    return result


In [15]:
res = domain_client.code.test_func(blocking=False)

Message Queued Successfully !
Message Received Successfully !


In [16]:
res.result

CONTEXT UID: 30f69842bea44f5c9f669a415787ad40
LOG_ID: 9b0d61214ae944afb37a05e48bacf594
LOOGER: <syft.service.log.log.SyftLogger object at 0x7f3f1beafc50>
Definition Complete
{'code_item': syft.service.code.user_code.UserCode, 'kwargs': {}, 'logger': <syft.service.log.log.SyftLogger object at 0x7f3f1beafc50>, 'stdout_': <ipykernel.iostream.OutStream object at 0x7f3f4f331660>, 'stderr_': <ipykernel.iostream.OutStream object at 0x7f3f4f331630>, 'stdout': <_io.StringIO object at 0x7f3f284237f0>, 'stderr': <_io.StringIO object at 0x7f3f28423760>, 'result': None, 'user_func_test_func_67d3b5eaf0c0bf6b5a602d359daecc86a7a74053490ec37ae08e71360587c870_530c8340dd99ba84b9533f27a22b96782da820ff5b391782cdd0c5bae62dd9f0': <function user_func_test_func_67d3b5eaf0c0bf6b5a602d359daecc86a7a74053490ec37ae08e71360587c870_530c8340dd99ba84b9533f27a22b96782da820ff5b391782cdd0c5bae62dd9f0 at 0x7f3f1bd16c00>, 'evil_string': 'user_func_test_func_67d3b5eaf0c0bf6b5a602d359daecc86a7a74053490ec37ae08e71360587c870_53

In [30]:
res.status

<Status.CREATED: 'created'>

In [28]:
domain_client.api.services.log.get_all()[1].stdout

'Start\n'

In [ ]:
log = domain_client.api.services.log.get(res.log_id)
log, log.stdout

AttributeError: 'NoneType' object has no attribute 'stdout'

In [ ]:
res.log_id, type(res.log_id)

(<UID: e364a4f64e594466bedfcf65b85e8099>, syft.types.uid.UID)

In [ ]:
test_func_uc = domain_client.code[-1]

In [ ]:
test_func_uc.byte_code

<code object <module> at 0x7f7ce5c44fa0, file "<string>", line 1>

In [ ]:
exec(test_func_uc.byte_code)
# exec(test_func_uc.byte_code)

In [ ]:
dir()

['In',
 'Out',
 '_',
 '_10',
 '_11',
 '_12',
 '_14',
 '_5',
 '_6',
 '_8',
 '_9',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'domain_client',
 'execute_byte_code',
 'exit',
 'get_ipython',
 'log',
 'node',
 'open',
 'quit',
 'res',
 'sy',
 'test_func',
 'test_func_uc',
 'user_func_test_func_67d3b5eaf0c0bf6b5a602d359daecc86a7a74053490ec37ae08e71360587c870_530c8340dd99ba84b9533f27a22b96782da820ff5b391782cdd0c5bae62dd9f0']

In [ ]:
test_func_uc.unique_func_name

'user_func_test_func_67d3b5eaf0c0bf6b5a602d359daecc86a7a74053490ec37ae08e71360587c870_530c8340dd99ba84b9533f27a22b96782da820ff5b391782cdd0c5bae62dd9f0'

In [ ]:
import logging
logger = logging.getLogger()

In [ ]:
kwargs={}
eval(f'{test_func_uc.unique_func_name}(**kwargs)')

In [ ]:
logger.debug("start")

In [ ]:
locals()['logger']

<RootLogger root (WARNING)>

In [ ]:
res = domain_client.code.call_async(uid=test_func_uc.id)

In [ ]:
import zmq
context = zmq.Context()
sub = context.socket(zmq.SUB)


In [ ]:
import ast
from syft.service.code.code_parse import GlobalsVisitor
from syft.service.code.unparse import unparse
from typing import List
def process_code(
    raw_code: str,
    func_name: str,
    original_func_name: str,
    input_kwargs: List[str],
) -> str:
    tree = ast.parse(raw_code)

    # check there are no globals
    v = GlobalsVisitor()
    v.visit(tree)

    f = tree.body[0]
    f.decorator_list = []

    keywords = [ast.keyword(arg=i, value=[ast.Name(id=i)]) for i in input_kwargs]
    call_stmt = ast.Assign(
        targets=[ast.Name(id="result")],
        value=ast.Call(
            func=ast.Name(id=original_func_name), args=[], keywords=keywords
        ),
        lineno=0,
    )

    return_stmt = ast.Return(value=ast.Name(id="result"))
    new_body = tree.body + [call_stmt, return_stmt]

    wrapper_function = ast.FunctionDef(
        name=func_name,
        args=f.args,
        body=new_body,
        decorator_list=[],
        returns=None,
        lineno=0,
    )

    return unparse(wrapper_function), wrapper_function

In [ ]:
from syft.client.api import NodeIdentity

input_kwargs = test_func.input_policy_init_kwargs
node_view_workaround = False
for k in input_kwargs.keys():
    if isinstance(k, NodeIdentity):
        node_view_workaround = True

if not node_view_workaround:
    input_keys = list(input_kwargs.keys())
else:
    input_keys = []
    for d in input_kwargs.values():
        input_keys += d.keys()
            
code_string, ast_obj = process_code(
    raw_code=test_func.code,
    func_name='final_test_func',
    original_func_name='test_func',
    input_kwargs=input_keys
)

In [ ]:
print(code_string)

def final_test_func():

    def test_func():
        import time
        logger.print('Start')
        time.sleep(1)
        return 1
    result = test_func()
    return result


In [ ]:
import ast
ast_obj = ast.Assign(targets=[ast.Name(id = 'logger')], value=ast.Name(id='logger'), lineno=0)

In [ ]:
print(ast.dump(ast_obj, indent=4))

FunctionDef(
    name='final_test_func',
    args=arguments(
        posonlyargs=[],
        args=[],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
    body=[
        FunctionDef(
            name='test_func',
            args=arguments(
                posonlyargs=[],
                args=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Import(
                    names=[
                        alias(name='time')]),
                Expr(
                    value=Call(
                        func=Attribute(
                            value=Name(id='logger'),
                            attr='print',
                            ctx=Load()),
                        args=[
                            Constant(value='Start')],
                        keywords=[])),
                Expr(
                    value=Call(
                        func=Attribute(
                        

In [ ]:
unparse(ast_obj)

'logger = logger'